<a href="https://colab.research.google.com/github/Lee-geun-bum/polars/blob/main/polars1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dfply

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.4/612.4 kB 33.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from dfply import *

# * filter , select , mutate

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "id":    [1, 1, 2, 2, 3],
    "group": ["A","A","B","B","A"],
    "x":     [10, 20, 30, 40, 50],
    "y":     [ 1,  2,  3,  4,  5]
})


In [ ]:
df

,id,group,x,y
0,1,A,10,1
1,1,A,20,2
2,2,B,30,3
3,2,B,40,4
4,3,A,50,5


In [ ]:
import numpy as np

res_pandas = (
    df
    # mutate: ratio = x / (y+1), z = x + y
    .assign(
        ratio = df["x"] / (df["y"] + 1),
        z     = lambda d: d["x"] + d["y"]
    )
    # filter: ratio >= 7.5
    .query("ratio >= 7.5")
    # select: id, group, ratio
    [["id","group","ratio"]]
    .sort_values(["id","group"])
)
print(res_pandas)


   id group     ratio
2   2     B  7.500000
3   2     B  8.000000
4   3     A  8.333333


In [ ]:
from dfply import *

res_dfply = (
    df
    >> mutate(ratio = X.x / (X.y + 1),
              z     = X.x + X.y)
    >> mask(X.ratio >= 7.5)         # dplyr의 filter()
    >> select(X.id, X.group, X.ratio)
    >> arrange(X.id, X.group)
)
print(res_dfply)


   id group     ratio
2   2     B  7.500000
3   2     B  8.000000
4   3     A  8.333333


In [ ]:
df

,id,group,x,y
0,1,A,10,1
1,1,A,20,2
2,2,B,30,3
3,2,B,40,4
4,3,A,50,5


# group_by, summarise

In [ ]:
df = pd.DataFrame({
    "dept": ["A","A","A","B","B","C"],
    "score": [80, 90, 85, 70, 60, 95],
    "hours": [5, 8, 7, 4, 3, 9]
})

df

,dept,score,hours
0,A,80,5
1,A,90,8
2,A,85,7
3,B,70,4
4,B,60,3
5,C,95,9


In [ ]:
res_pandas = (
    df
    .groupby("dept", as_index=False)
    .agg(
        mean_score=("score","mean"),
        total_hours=("hours","sum"),
        n=("score","count")
    )
)
print(res_pandas)


  dept  mean_score  total_hours  n
0    A        85.0           20  3
1    B        65.0            7  2
2    C        95.0            9  1


In [ ]:
df = pl.DataFrame({
    "dept": ["A","A","A","B","B","C"],
    "score": [80, 90, 85, 70, 60, 95],
    "hours": [5, 8, 7, 4, 3, 9]
})


res_polas = (
    df
    .group_by("dept")
    .agg(
        pl.col("score").mean().alias("mean_score"),
        pl.col("hours").sum().alias("total_hours"),
        pl.col("score").count().alias("n")
    )
    .sort("n", descending = True)
)


res_polas

dept,mean_score,total_hours,n
str,f64,i64,u32
"""A""",85.0,20,3
"""B""",65.0,7,2
"""C""",95.0,9,1


In [ ]:
# res_dfply = (
#     df
#     >> group_by(X.dept)
#     >> summarize(
#         mean_score = X.score.mean(),
#         total_hours = X.hours.sum(),
#         n = X.score.count()
#     )
# )
# print(res_dfply)


In [ ]:
 ## polas. ... 맘에든다 이놈으로

In [ ]:
# df %>%
#   filter(age > 20, gender == "F") %>%
#   mutate(income_ratio = income / mean(income, na.rm=TRUE)) %>%
#   group_by(region) %>%
#   summarise(avg_ratio = mean(income_ratio))


In [ ]:
# import polars as pl

# df = (
#     pl.read_csv("data.csv")
#       .filter((pl.col("age") > 20) & (pl.col("gender") == "F"))
#       .with_columns(
#           (pl.col("income") / pl.col("income").mean()).alias("income_ratio")
#       )
#       .group_by("region")
#       .agg(pl.col("income_ratio").mean().alias("avg_ratio"))
# )


In [ ]:
import polars as pl


# 예제 데이터 생성
df = pl.DataFrame({
    "student": ["A","B","C","D","E","F","G","H","I","J"],
    "grade":   [1,1,1,2,2,2,3,3,3,3],
    "subject": ["math","math","math","eng","eng","eng","sci","sci","sci","sci"],
    "score":   [88, 92, 75, 60, 82, 90, 78, 95, 100, 65]
})

df


student,grade,subject,score
str,i64,str,i64
"""A""",1,"""math""",88
"""B""",1,"""math""",92
"""C""",1,"""math""",75
"""D""",2,"""eng""",60
"""E""",2,"""eng""",82
"""F""",2,"""eng""",90
"""G""",3,"""sci""",78
"""H""",3,"""sci""",95
"""I""",3,"""sci""",100


In [ ]:
# 전처리: group_by + filter + mutate + summarise
result = (
    df
    # 1️⃣ 각 과목별 평균 계산
    .with_columns(
        pl.col("score").mean().over("subject").alias("mean_subject")
    )
    # 2️⃣ 과목 평균보다 높은 학생만 선택
    .filter(pl.col("score") > pl.col("mean_subject"))
    # 3️⃣ 학생의 과목 내 점수 비율 계산
    .with_columns(
        (pl.col("score") / pl.col("mean_subject")).alias("ratio")
    )
    # 4️⃣ 학년(grade)별 평균 비율 구하기
    .group_by("grade")
    .agg(
        pl.col("ratio").mean().alias("avg_ratio"),
        pl.count().alias("n_students")
    )
    # 5️⃣ 평균 비율이 높은 순으로 정렬
    .sort("avg_ratio", descending=True)
)

print(result)

shape: (3, 3)
┌───────┬───────────┬────────────┐
│ grade ┆ avg_ratio ┆ n_students │
│ ---   ┆ ---       ┆ ---        │
│ i64   ┆ f64       ┆ u32        │
╞═══════╪═══════════╪════════════╡
│ 3     ┆ 1.153846  ┆ 2          │
│ 2     ┆ 1.112069  ┆ 2          │
│ 1     ┆ 1.058824  ┆ 2          │
└───────┴───────────┴────────────┘


/tmp/ipython-input-453485368.py:18: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("n_students")


 ** 위아래 차이 확인

In [ ]:
# 전처리: group_by + filter + mutate + summarise
result = (
    df
    # 1️⃣ 각 과목별 평균 계산
    .with_columns(
        pl.col("score").mean().over("subject").alias("mean_subject")
    )
    # 2️⃣ 과목 평균보다 높은 학생만 선택
    .filter(pl.col("score") > pl.col("mean_subject"))
    # 3️⃣ 학생의 과목 내 점수 비율 계산
    .with_columns(
        (pl.col("score") / pl.col("mean_subject")).alias("ratio")
    )
    # 4️⃣ 학년(grade)별 평균 비율 구하기
    .with_columns(
        pl.col("ratio").mean().over("grade").alias("avg_ratio"),
        pl.len().over("grade").alias("n_students")
    )
    # 5️⃣ 평균 비율이 높은 순으로 정렬
    .sort("avg_ratio", descending=True)
)

result_pd = result.to_pandas()
print(result_pd)

  student  grade subject  score  mean_subject     ratio  avg_ratio  n_students
0       H      3     sci     95     84.500000  1.124260   1.153846           2
1       I      3     sci    100     84.500000  1.183432   1.153846           2
2       E      2     eng     82     77.333333  1.060345   1.112069           2
3       F      2     eng     90     77.333333  1.163793   1.112069           2
4       A      1    math     88     85.000000  1.035294   1.058824           2
5       B      1    math     92     85.000000  1.082353   1.058824           2


In [ ]:
  ## 개꿀함수
  ## groupby 에 summarise 한거 join 할때 한번에 유용할듯, 근데 전체 그거에 대해서임 na로 자동으로는 안오네

    # .with_columns(
    #     pl.col("score").mean().over("subject").alias("mean_subject")
    # )

In [ ]:
# import polars as pl

# # 모든 칼럼 선택
# selected_df = pl_df.select(pl.col("*"))
# selected_df = pl_df.select(pl.all())

# pl_df.select(pl.col(pl.Int64)) # 데이터 타입과 타입의 크기에 따라 선택 가능

# import polars.selectors as cs

# # int64, int32인지 구분하지 않고 정수형 칼럼을 선택하고 싶을 때
# pl_out_with_int = pl_df.select(cs.integer())

# # int64, int16, float32인지 구분하지 않고 실수형 칼럼과 문자열 칼럼을 선택하고 싶을 때
# pl_out_with_num = pl_df.select(cs.numeric(), cs.string())

# # 칼럼 이름에 정규식을 적용해서 선택(_encoded로 끝나는 칼럼)
# pl_df.select(pl.col("^.*_encoded$"))

# # 일부 열만 제외
# pl_df.select(pl.exclude("name", "private_number"))